In [6]:
import bs4
import requests
import urllib.request
from selenium import webdriver
from urllib import parse
import time
import re
import pandas as pd

def kakaopage_crawling(search_name):

    tmp = search_name.split('[')[0]
    search_name_nonspace=search_name.replace(" ","")

    url = "https://page.kakao.com/search?word=" + parse.quote(tmp)

    r=requests.get(url)
    c=r.content
    bs_obj = bs4.BeautifulSoup(c, "html.parser")

    boxes = bs_obj.findAll("div", {"class": "css-c09e5i"})

    
    for box in boxes:
        is_novel=box.find("div",{"class":"css-vurnku"}).text
        if("소설" in is_novel):
            parent=box.parent.parent
            title=parent.find("div", {"class": "text-ellipsis css-11602e0"}).text.replace(" ","")
            #if("단행본" in title):
            #    continue
            if(search_name_nonspace not in title):
                continue
            else:
                main=parent.parent.parent
                monopoly=True
        else:
            continue
    title=main.find("div", {"class": "text-ellipsis css-11602e0"}).text

    if '[' in title:
        print("작품이름 : " + title)
        #exclusive=title.split('[')[1][:-1]
        #print("독점여부 : " + exclusive)
    else:
        print("작품이름 : " + title)
        #exclusive='미독점'
        #print("독점여부 : " + exclusive)

    #print("감상인원 : " + main.find('div',{'class','css-zlhhis'}).text)
    genre=main.findAll('div',{'class','css-vurnku'})[1].text
    print("장르 : " + genre)
    author=main.find('div',{'class','text-ellipsis css-1pysja1'}).text
    print("작가 : " + author)
    #print("작가 : " + main.find('div',{'class','css-text-ellipsis css-1pysja1'}).text)

    complete=main.find('div',{'class','text-ellipsis css-1eorkc4'}).text

    if complete!="완결":
        complete='미완결'
        print("완결여부 : "+ complete)
        #print("최근업데이트일 : " + latest.text.replace(" 업데이트",""))

    else:
        print("완결여부 : " + complete)

    img_src=main.find('div',{'class','css-xw0ejb'}).find('img').attrs['data-src']
    print("이미지 : " + img_src)


    age=main.find("div", {"class": "css-fe9s02"}).find('img')
    if age==None:
        age="전체이용가"
        print(age)
    else:
        age="15세 이용가"
        print(age)


    pages=set()

    for link in main.findAll("a", href = re.compile('^(/home)((?!:).)*$')):
        if 'href' in link.attrs: # 위에서 찾은 link에 href 속성이 있는지 확인   
            if link.attrs['href'] not in pages: # 새로운 페이지인지 확인                 
                newPage = link.attrs['href']

    #기다무소설or 소설
    #[단행본] X
    #[닥터최태수 or 닥터 최태수 포함]

    #작품의 링크
    url="https://page.kakao.com" + newPage 
    href=url
    driver.get(url)
    bs_obj = bs4.BeautifulSoup(driver.page_source, "html.parser")
    time.sleep(1)
    #pub_date=driver.find_element_by_xpath('//*[@id="root"]/div[3]/div/div/div[3]/ul/li[1]/div[2]/dl/dd').text
    #print("출간일 : " + pub_date)
    #time.sleep(0.2)

    #if complete!="완결":
    #    update_days = driver.find_element_by_xpath('//*[@id="root"]/div[3]/div/div/div[1]/div[2]/div[2]/div[1]/p[1]')
    #    print("연재 요일 : " + update_days.text.replace(" 연재",""))

    #else:
    #    print("연재 요일 : 연재 종료")
    #time.sleep(0.2)

    total=driver.find_element_by_xpath('//*[@id="root"]/div[3]/div/div/div[3]/div[1]/span[1]')
    total=total.text.replace('전체(',"").replace(')',"")
    print("연재 수 : "+ total)
    time.sleep(0.2)

    pub_date=driver.find_element_by_xpath('//*[@id="root"]/div[3]/div/div/div[3]/ul/li[1]/div[2]/dl/dd').text
    print("최초 업데이트일 : " + pub_date)
    time.sleep(0.2)

    time.sleep(2)

    driver.find_element_by_xpath('//*[@id="root"]/div[3]/div/div/div[1]/div[2]/div[3]/div[2]/button[1]').click()
    #driver.set_window_size(1400,1000)
    publisher=None
    try:
        publisher=driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div/div/table/tbody/tr[1]/td[2]/div[3]/div[2]').text
    except:
        pass

    if publisher==None:
        publisher="정보 없음"
    print("출판사 : " + publisher)
    time.sleep(0.2)


    intro=driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div/div/table/tbody/tr[1]/td[2]/div[1]').text
    print("작품소개\n" + "----------------------------------------\n" + intro + "\n----------------------------------------")
    time.sleep(0.2)

    #driver2.quit()

    info_list = [title, genre, author, complete, img_src, age, total, pub_date, publisher, intro,href]
    
    return info_list
    #최근 작품등록일 selenium한번더 열어서.
    """
    
    

    """

In [16]:
import bs4
import requests
from urllib import parse
import selenium
import csv
import re
from selenium import webdriver
import time

def kakaopage_crawling(search_name):
    
    search_name=search_name.replace('[BL]','').replace('[GL]','').replace('[완결]','').replace('(완결)','').replace('[연재]','').replace('(개정판)','').replace('[단행본]','').replace('[개정판]','')
    if '?' in search_name : 
        tmp=search_name.split('?')[0]
    
    elif '!' in search_name  :
        tmp=search_name.split('!')[0]
    
    elif '-' in search_name  :
        tmp=search_name.split('-')[0]
    elif '(' in search_name  :
        tmp=search_name.split('(')[0]
    elif ':' in search_name  :
        tmp=search_name.split(':')[0]    
    elif ',' in search_name  :
        tmp=search_name.split(',')[1]  
    elif '&' in search_name  :
        tmp=search_name.split('&')[0]  
    else :
        tmp = search_name.split('[')[0]

    print(tmp)
    search_name_nonspace=tmp.replace(" ","")

    url = "https://page.kakao.com/search?word=" + parse.quote(tmp)

    r=requests.get(url)
    c=r.content
    bs_obj = bs4.BeautifulSoup(c, "html.parser")

    boxes = bs_obj.findAll("div", {"class": "css-c09e5i"})
    
    for box in boxes:
        is_novel=box.find("div",{"class":"css-vurnku"}).text
        if("소설" in is_novel):
            parent=box.parent.parent
            title=parent.find("div", {"class": "text-ellipsis css-11602e0"}).text.replace(" ","")
            #if("단행본" in title):
            #    continue
            if(search_name_nonspace not in title):
                continue
            else:
                main=parent.parent.parent
                monopoly=True
        else:
            continue
    
    title=main.find("div", {"class": "text-ellipsis css-11602e0"}).text

    if '[' in title:
        print("작품이름 : " + title)
        #exclusive=title.split('[')[1][:-1]
        #print("독점여부 : " + exclusive)
    else:
        print("작품이름 : " + title)
        #exclusive='미독점'
        #print("독점여부 : " + exclusive)

    #print("감상인원 : " + main.find('div',{'class','css-zlhhis'}).text)
    genre=main.findAll('div',{'class','css-vurnku'})[1].text
    print("장르 : " + genre)
    author=main.find('div',{'class','text-ellipsis css-1pysja1'}).text
    print("작가 : " + author)
    #print("작가 : " + main.find('div',{'class','css-text-ellipsis css-1pysja1'}).text)

    complete=main.find('div',{'class','text-ellipsis css-1eorkc4'}).text

    if complete!="완결":
        complete='미완결'
        print("완결여부 : "+ complete)
        #print("최근업데이트일 : " + latest.text.replace(" 업데이트",""))

    else:
        print("완결여부 : " + complete)

    img_src=main.find('div',{'class','css-xw0ejb'}).find('img').attrs['data-src']
    print("이미지 : " + img_src)


    age=main.find("div", {"class": "css-fe9s02"}).find('img')
    if age==None:
        age="전체이용가"
        print(age)
    else:
        age="15세 이용가"
        print(age)


    pages=set()

    for link in main.findAll("a", href = re.compile('^(/home)((?!:).)*$')):
        if 'href' in link.attrs: # 위에서 찾은 link에 href 속성이 있는지 확인   
            if link.attrs['href'] not in pages: # 새로운 페이지인지 확인                 
                newPage = link.attrs['href']

    #기다무소설or 소설
    #[단행본] X
    #[닥터최태수 or 닥터 최태수 포함]

    #작품의 링크
    url="https://page.kakao.com" + newPage +"&orderby=asc"
    href=url
    driver.get(url)
    bs_obj = bs4.BeautifulSoup(driver.page_source, "html.parser")
    time.sleep(1)
    #pub_date=driver.find_element_by_xpath('//*[@id="root"]/div[3]/div/div/div[3]/ul/li[1]/div[2]/dl/dd').text
    #print("출간일 : " + pub_date)
    #time.sleep(0.2)

    #if complete!="완결":
    #    update_days = driver.find_element_by_xpath('//*[@id="root"]/div[3]/div/div/div[1]/div[2]/div[2]/div[1]/p[1]')
    #    print("연재 요일 : " + update_days.text.replace(" 연재",""))

    #else:
    #    print("연재 요일 : 연재 종료")
    #time.sleep(0.2)
    
    
    total=driver.find_element_by_xpath('//*[@id="root"]/div[3]/div/div/div[3]/div[1]/span[1]')
    total=total.text.replace('전체(',"").replace(')',"")
    print("연재 수 : "+ total)
    time.sleep(0.2)

    pub_date=driver.find_element_by_xpath('//*[@id="root"]/div[3]/div/div/div[3]/ul/li[1]/div/dl/dd').text
    print("최초 업데이트일 : " + pub_date)
    time.sleep(0.2)

    time.sleep(2)

    driver.find_element_by_xpath('//*[@id="root"]/div[3]/div/div/div[1]/div[2]/div[3]/div[2]/button[1]').click()
    #driver.set_window_size(1400,1000)
    publisher=None
    try:
        publisher=driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div/div/table/tbody/tr[1]/td[2]/div[3]/div[2]').text
    except:
        pass

    if publisher==None:
        publisher="정보 없음"
    print("출판사 : " + publisher)
    time.sleep(0.2)


    intro=driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div/div/table/tbody/tr[1]/td[2]/div[1]').text
    print("작품소개\n" + "----------------------------------------\n" + intro + "\n----------------------------------------")
    time.sleep(0.2)

    #driver2.quit()

    info_list = [title.replace(u'\xf3','').replace(u'\u7448','').replace(u'\u2981','').replace(u'\u3e6e','').replace(u'\u586d','').replace(u'\u614f','').replace(u'\uff63','').replace(u'\uff62','').replace(u'\u2022','').replace(u'\u231f','').replace(u'\u231c','').replace(u'\xed','').replace(u'\u0165','').replace(u'\u0161','').replace(u'\xe1','').replace(u'\u2027','').replace(u'\u82e4','').replace(u'\u51e4','').replace(u'\u989c','').replace(u'\u971b','').replace(u'\u7978','').replace(u'\u6ce0','').replace(u'\u503e','').replace(u'\u98cb','').replace(u'\u22ef','').replace(u'\u554a','').replace(u'\u6720','').replace(u'\u71cf','').replace(u'\ufeff','').replace(u'\u5f37','').replace(u'\u630c','').replace(u'\u7681','').replace(u'\u512b','').replace(u'\u70e5','').replace(u'\u525f','').replace(u'\u3983','').replace(u'\u89c1','').replace(u'\u993b','').replace(u'\u5a7f','').replace(u'\u8d58','').replace(u'\u34e6','').replace(u'\u760b','').replace(u'\u7ecf','').replace(u'\u24b8','').replace(u'\xa6','').replace(u'\u3007','').replace(u'\u72ec','').replace(u'\u30fc','').replace(u'\xe9','').replace(u'\u30fb','').replace(u'\u9b47','').replace(u'\u68a6','').replace(u'\u8273','').replace(u'\u756b','').replace(u'\u65f6','').replace(u'\u2014','').replace(u'\u72bf','').replace(u'\u533b','').replace(u'\u25ac','').replace(u'\u5767','').replace(u'\u2219','').replace(u'\u7955','').replace(u'\u3f01','').replace(u'\xa9','').replace(u'\u5f72','').replace(u'\u95b6','').replace(u'\u25ab','').replace(u'\u2013','').replace(u'\u2609','').replace(u'\u880d','').replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 genre.replace(u'\xf3','').replace(u'\u7448','').replace(u'\u2981','').replace(u'\u3e6e','').replace(u'\u586d','').replace(u'\u614f','').replace(u'\uff63','').replace(u'\uff62','').replace(u'\u2022','').replace(u'\u231f','').replace(u'\u231c','').replace(u'\xed','').replace(u'\u0165','').replace(u'\u0161','').replace(u'\xe1','').replace(u'\u2027','').replace(u'\u82e4','').replace(u'\u51e4','').replace(u'\u989c','').replace(u'\u971b','').replace(u'\u7978','').replace(u'\u6ce0','').replace(u'\u503e','').replace(u'\u98cb','').replace(u'\u22ef','').replace(u'\u554a','').replace(u'\u6720','').replace(u'\u71cf','').replace(u'\ufeff','').replace(u'\u5f37','').replace(u'\u630c','').replace(u'\u7681','').replace(u'\u512b','').replace(u'\u70e5','').replace(u'\u525f','').replace(u'\u3983','').replace(u'\u89c1','').replace(u'\u993b','').replace(u'\u5a7f','').replace(u'\u8d58','').replace(u'\u34e6','').replace(u'\u760b','').replace(u'\u7ecf','').replace(u'\u24b8','').replace(u'\xa6','').replace(u'\u3007','').replace(u'\u72ec','').replace(u'\u30fc','').replace(u'\xe9','').replace(u'\u30fb','').replace(u'\u9b47','').replace(u'\u68a6','').replace(u'\u8273','').replace(u'\u756b','').replace(u'\u65f6','').replace(u'\u2014','').replace(u'\u72bf','').replace(u'\u533b','').replace(u'\u25ac','').replace(u'\u5767','').replace(u'\u2219','').replace(u'\u7955','').replace(u'\u3f01','').replace(u'\xa9','').replace(u'\u5f72','').replace(u'\u95b6','').replace(u'\u25ab','').replace(u'\u2013','').replace(u'\u2609','').replace(u'\u880d','').replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 author.replace(u'\xf3','').replace(u'\u7448','').replace(u'\u2981','').replace(u'\u3e6e','').replace(u'\u586d','').replace(u'\u614f','').replace(u'\uff63','').replace(u'\uff62','').replace(u'\u2022','').replace(u'\u231f','').replace(u'\u231c','').replace(u'\xed','').replace(u'\u0165','').replace(u'\u0161','').replace(u'\xe1','').replace(u'\u2027','').replace(u'\u82e4','').replace(u'\u51e4','').replace(u'\u989c','').replace(u'\u971b','').replace(u'\u7978','').replace(u'\u6ce0','').replace(u'\u503e','').replace(u'\u98cb','').replace(u'\u22ef','').replace(u'\u554a','').replace(u'\u6720','').replace(u'\u71cf','').replace(u'\ufeff','').replace(u'\u5f37','').replace(u'\u630c','').replace(u'\u7681','').replace(u'\u512b','').replace(u'\u70e5','').replace(u'\u525f','').replace(u'\u3983','').replace(u'\u89c1','').replace(u'\u993b','').replace(u'\u5a7f','').replace(u'\u8d58','').replace(u'\u34e6','').replace(u'\u760b','').replace(u'\u7ecf','').replace(u'\u24b8','').replace(u'\xa6','').replace(u'\u3007','').replace(u'\u72ec','').replace(u'\u30fc','').replace(u'\xe9','').replace(u'\u30fb','').replace(u'\u9b47','').replace(u'\u68a6','').replace(u'\u8273','').replace(u'\u756b','').replace(u'\u65f6','').replace(u'\u2014','').replace(u'\u72bf','').replace(u'\u533b','').replace(u'\u25ac','').replace(u'\u5767','').replace(u'\u2219','').replace(u'\u7955','').replace(u'\u3f01','').replace(u'\xa9','').replace(u'\u5f72','').replace(u'\u95b6','').replace(u'\u25ab','').replace(u'\u2013','').replace(u'\u2609','').replace(u'\u880d','').replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 complete.replace(u'\xf3','').replace(u'\u7448','').replace(u'\u2981','').replace(u'\u3e6e','').replace(u'\u586d','').replace(u'\u614f','').replace(u'\uff63','').replace(u'\uff62','').replace(u'\u2022','').replace(u'\u231f','').replace(u'\u231c','').replace(u'\xed','').replace(u'\u0165','').replace(u'\u0161','').replace(u'\xe1','').replace(u'\u2027','').replace(u'\u82e4','').replace(u'\u51e4','').replace(u'\u989c','').replace(u'\u971b','').replace(u'\u7978','').replace(u'\u6ce0','').replace(u'\u503e','').replace(u'\u98cb','').replace(u'\u22ef','').replace(u'\u554a','').replace(u'\u6720','').replace(u'\u71cf','').replace(u'\ufeff','').replace(u'\u5f37','').replace(u'\u630c','').replace(u'\u7681','').replace(u'\u512b','').replace(u'\u70e5','').replace(u'\u525f','').replace(u'\u3983','').replace(u'\u89c1','').replace(u'\u993b','').replace(u'\u5a7f','').replace(u'\u8d58','').replace(u'\u34e6','').replace(u'\u760b','').replace(u'\u7ecf','').replace(u'\u24b8','').replace(u'\xa6','').replace(u'\u3007','').replace(u'\u72ec','').replace(u'\u30fc','').replace(u'\xe9','').replace(u'\u30fb','').replace(u'\u9b47','').replace(u'\u68a6','').replace(u'\u8273','').replace(u'\u756b','').replace(u'\u65f6','').replace(u'\u2014','').replace(u'\u72bf','').replace(u'\u533b','').replace(u'\u25ac','').replace(u'\u5767','').replace(u'\u2219','').replace(u'\u7955','').replace(u'\u3f01','').replace(u'\xa9','').replace(u'\u5f72','').replace(u'\u95b6','').replace(u'\u25ab','').replace(u'\u2013','').replace(u'\u2609','').replace(u'\u880d','').replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 img_src.replace(u'\xf3','').replace(u'\u7448','').replace(u'\u2981','').replace(u'\u3e6e','').replace(u'\u586d','').replace(u'\u614f','').replace(u'\uff63','').replace(u'\uff62','').replace(u'\u2022','').replace(u'\u231f','').replace(u'\u231c','').replace(u'\xed','').replace(u'\u0165','').replace(u'\u0161','').replace(u'\xe1','').replace(u'\u2027','').replace(u'\u82e4','').replace(u'\u51e4','').replace(u'\u989c','').replace(u'\u971b','').replace(u'\u7978','').replace(u'\u6ce0','').replace(u'\u503e','').replace(u'\u98cb','').replace(u'\u22ef','').replace(u'\u554a','').replace(u'\u6720','').replace(u'\u71cf','').replace(u'\ufeff','').replace(u'\u5f37','').replace(u'\u630c','').replace(u'\u7681','').replace(u'\u512b','').replace(u'\u70e5','').replace(u'\u525f','').replace(u'\u3983','').replace(u'\u89c1','').replace(u'\u993b','').replace(u'\u5a7f','').replace(u'\u8d58','').replace(u'\u34e6','').replace(u'\u760b','').replace(u'\u7ecf','').replace(u'\u24b8','').replace(u'\xa6','').replace(u'\u3007','').replace(u'\u72ec','').replace(u'\u30fc','').replace(u'\xe9','').replace(u'\u30fb','').replace(u'\u9b47','').replace(u'\u68a6','').replace(u'\u8273','').replace(u'\u756b','').replace(u'\u65f6','').replace(u'\u2014','').replace(u'\u72bf','').replace(u'\u533b','').replace(u'\u25ac','').replace(u'\u5767','').replace(u'\u2219','').replace(u'\u7955','').replace(u'\u3f01','').replace(u'\xa9','').replace(u'\u5f72','').replace(u'\u95b6','').replace(u'\u25ab','').replace(u'\u2013','').replace(u'\u2609','').replace(u'\u880d','').replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 age.replace(u'\xf3','').replace(u'\u7448','').replace(u'\u2981','').replace(u'\u3e6e','').replace(u'\u586d','').replace(u'\u614f','').replace(u'\uff63','').replace(u'\uff62','').replace(u'\u2022','').replace(u'\u231f','').replace(u'\u231c','').replace(u'\xed','').replace(u'\u0165','').replace(u'\u0161','').replace(u'\xe1','').replace(u'\u2027','').replace(u'\u82e4','').replace(u'\u51e4','').replace(u'\u989c','').replace(u'\u971b','').replace(u'\u7978','').replace(u'\u6ce0','').replace(u'\u503e','').replace(u'\u98cb','').replace(u'\u22ef','').replace(u'\u554a','').replace(u'\u6720','').replace(u'\u71cf','').replace(u'\ufeff','').replace(u'\u5f37','').replace(u'\u630c','').replace(u'\u7681','').replace(u'\u512b','').replace(u'\u70e5','').replace(u'\u525f','').replace(u'\u3983','').replace(u'\u89c1','').replace(u'\u993b','').replace(u'\u5a7f','').replace(u'\u8d58','').replace(u'\u34e6','').replace(u'\u760b','').replace(u'\u7ecf','').replace(u'\u24b8','').replace(u'\xa6','').replace(u'\u3007','').replace(u'\u72ec','').replace(u'\u30fc','').replace(u'\xe9','').replace(u'\u30fb','').replace(u'\u9b47','').replace(u'\u68a6','').replace(u'\u8273','').replace(u'\u756b','').replace(u'\u65f6','').replace(u'\u2014','').replace(u'\u72bf','').replace(u'\u533b','').replace(u'\u25ac','').replace(u'\u5767','').replace(u'\u2219','').replace(u'\u7955','').replace(u'\u3f01','').replace(u'\xa9','').replace(u'\u5f72','').replace(u'\u95b6','').replace(u'\u25ab','').replace(u'\u2013','').replace(u'\u2609','').replace(u'\u880d','').replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 total.replace(u'\xf3','').replace(u'\u7448','').replace(u'\u2981','').replace(u'\u3e6e','').replace(u'\u586d','').replace(u'\u614f','').replace(u'\uff63','').replace(u'\uff62','').replace(u'\u2022','').replace(u'\u231f','').replace(u'\u231c','').replace(u'\xed','').replace(u'\u0165','').replace(u'\u0161','').replace(u'\xe1','').replace(u'\u2027','').replace(u'\u82e4','').replace(u'\u51e4','').replace(u'\u989c','').replace(u'\u971b','').replace(u'\u7978','').replace(u'\u6ce0','').replace(u'\u503e','').replace(u'\u98cb','').replace(u'\u22ef','').replace(u'\u554a','').replace(u'\u6720','').replace(u'\u71cf','').replace(u'\ufeff','').replace(u'\u5f37','').replace(u'\u630c','').replace(u'\u7681','').replace(u'\u512b','').replace(u'\u70e5','').replace(u'\u525f','').replace(u'\u3983','').replace(u'\u89c1','').replace(u'\u993b','').replace(u'\u5a7f','').replace(u'\u8d58','').replace(u'\u34e6','').replace(u'\u760b','').replace(u'\u7ecf','').replace(u'\u24b8','').replace(u'\xa6','').replace(u'\u3007','').replace(u'\u72ec','').replace(u'\u30fc','').replace(u'\xe9','').replace(u'\u30fb','').replace(u'\u9b47','').replace(u'\u68a6','').replace(u'\u8273','').replace(u'\u756b','').replace(u'\u65f6','').replace(u'\u2014','').replace(u'\u72bf','').replace(u'\u533b','').replace(u'\u25ac','').replace(u'\u5767','').replace(u'\u2219','').replace(u'\u7955','').replace(u'\u3f01','').replace(u'\xa9','').replace(u'\u5f72','').replace(u'\u95b6','').replace(u'\u25ab','').replace(u'\u2013','').replace(u'\u2609','').replace(u'\u880d','').replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 pub_date.replace(u'\xf3','').replace(u'\u7448','').replace(u'\u2981','').replace(u'\u3e6e','').replace(u'\u586d','').replace(u'\u614f','').replace(u'\uff63','').replace(u'\uff62','').replace(u'\u2022','').replace(u'\u231f','').replace(u'\u231c','').replace(u'\xed','').replace(u'\u0165','').replace(u'\u0161','').replace(u'\xe1','').replace(u'\u2027','').replace(u'\u82e4','').replace(u'\u51e4','').replace(u'\u989c','').replace(u'\u971b','').replace(u'\u7978','').replace(u'\u6ce0','').replace(u'\u503e','').replace(u'\u98cb','').replace(u'\u22ef','').replace(u'\u554a','').replace(u'\u6720','').replace(u'\u71cf','').replace(u'\ufeff','').replace(u'\u5f37','').replace(u'\u630c','').replace(u'\u7681','').replace(u'\u512b','').replace(u'\u70e5','').replace(u'\u525f','').replace(u'\u3983','').replace(u'\u89c1','').replace(u'\u993b','').replace(u'\u5a7f','').replace(u'\u8d58','').replace(u'\u34e6','').replace(u'\u760b','').replace(u'\u7ecf','').replace(u'\u24b8','').replace(u'\xa6','').replace(u'\u3007','').replace(u'\u72ec','').replace(u'\u30fc','').replace(u'\xe9','').replace(u'\u30fb','').replace(u'\u9b47','').replace(u'\u68a6','').replace(u'\u8273','').replace(u'\u756b','').replace(u'\u65f6','').replace(u'\u2014','').replace(u'\u72bf','').replace(u'\u533b','').replace(u'\u25ac','').replace(u'\u5767','').replace(u'\u2219','').replace(u'\u7955','').replace(u'\u3f01','').replace(u'\xa9','').replace(u'\u5f72','').replace(u'\u95b6','').replace(u'\u25ab','').replace(u'\u2013','').replace(u'\u2609','').replace(u'\u880d','').replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 publisher.replace(u'\xf3','').replace(u'\u7448','').replace(u'\u2981','').replace(u'\u3e6e','').replace(u'\u586d','').replace(u'\u614f','').replace(u'\uff63','').replace(u'\uff62','').replace(u'\u2022','').replace(u'\u231f','').replace(u'\u231c','').replace(u'\xed','').replace(u'\u0165','').replace(u'\u0161','').replace(u'\xe1','').replace(u'\u2027','').replace(u'\u82e4','').replace(u'\u51e4','').replace(u'\u989c','').replace(u'\u971b','').replace(u'\u7978','').replace(u'\u6ce0','').replace(u'\u503e','').replace(u'\u98cb','').replace(u'\u22ef','').replace(u'\u554a','').replace(u'\u6720','').replace(u'\u71cf','').replace(u'\ufeff','').replace(u'\u5f37','').replace(u'\u630c','').replace(u'\u7681','').replace(u'\u512b','').replace(u'\u70e5','').replace(u'\u525f','').replace(u'\u3983','').replace(u'\u89c1','').replace(u'\u993b','').replace(u'\u5a7f','').replace(u'\u8d58','').replace(u'\u34e6','').replace(u'\u760b','').replace(u'\u7ecf','').replace(u'\u24b8','').replace(u'\xa6','').replace(u'\u3007','').replace(u'\u72ec','').replace(u'\u30fc','').replace(u'\xe9','').replace(u'\u30fb','').replace(u'\u9b47','').replace(u'\u68a6','').replace(u'\u8273','').replace(u'\u756b','').replace(u'\u65f6','').replace(u'\u2014','').replace(u'\u72bf','').replace(u'\u533b','').replace(u'\u25ac','').replace(u'\u5767','').replace(u'\u2219','').replace(u'\u7955','').replace(u'\u3f01','').replace(u'\xa9','').replace(u'\u5f72','').replace(u'\u95b6','').replace(u'\u25ab','').replace(u'\u2013','').replace(u'\u2609','').replace(u'\u880d','').replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 intro.replace(u'\xf3','').replace(u'\u7448','').replace(u'\u2981','').replace(u'\u3e6e','').replace(u'\u586d','').replace(u'\u614f','').replace(u'\uff63','').replace(u'\uff62','').replace(u'\u2022','').replace(u'\u231f','').replace(u'\u231c','').replace(u'\xed','').replace(u'\u0165','').replace(u'\u0161','').replace(u'\xe1','').replace(u'\u2027','').replace(u'\u82e4','').replace(u'\u51e4','').replace(u'\u989c','').replace(u'\u971b','').replace(u'\u7978','').replace(u'\u6ce0','').replace(u'\u503e','').replace(u'\u98cb','').replace(u'\u22ef','').replace(u'\u554a','').replace(u'\u6720','').replace(u'\u71cf','').replace(u'\ufeff','').replace(u'\u5f37','').replace(u'\u630c','').replace(u'\u7681','').replace(u'\u512b','').replace(u'\u70e5','').replace(u'\u525f','').replace(u'\u3983','').replace(u'\u89c1','').replace(u'\u993b','').replace(u'\u5a7f','').replace(u'\u8d58','').replace(u'\u34e6','').replace(u'\u760b','').replace(u'\u7ecf','').replace(u'\u24b8','').replace(u'\xa6','').replace(u'\u3007','').replace(u'\u72ec','').replace(u'\u30fc','').replace(u'\xe9','').replace(u'\u30fb','').replace(u'\u9b47','').replace(u'\u68a6','').replace(u'\u8273','').replace(u'\u756b','').replace(u'\u65f6','').replace(u'\u2014','').replace(u'\u72bf','').replace(u'\u533b','').replace(u'\u25ac','').replace(u'\u5767','').replace(u'\u2219','').replace(u'\u7955','').replace(u'\u3f01','').replace(u'\xa9','').replace(u'\u5f72','').replace(u'\u95b6','').replace(u'\u25ab','').replace(u'\u2013','').replace(u'\u2609','').replace(u'\u880d','').replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 href.replace(u'\xf3','').replace(u'\u7448','').replace(u'\u2981','').replace(u'\u3e6e','').replace(u'\u586d','').replace(u'\u614f','').replace(u'\uff63','').replace(u'\uff62','').replace(u'\u2022','').replace(u'\u231f','').replace(u'\u231c','').replace(u'\xed','').replace(u'\u0165','').replace(u'\u0161','').replace(u'\xe1','').replace(u'\u2027','').replace(u'\u82e4','').replace(u'\u51e4','').replace(u'\u989c','').replace(u'\u971b','').replace(u'\u7978','').replace(u'\u6ce0','').replace(u'\u503e','').replace(u'\u98cb','').replace(u'\u22ef','').replace(u'\u554a','').replace(u'\u6720','').replace(u'\u71cf','').replace(u'\ufeff','').replace(u'\u5f37','').replace(u'\u630c','').replace(u'\u7681','').replace(u'\u512b','').replace(u'\u70e5','').replace(u'\u525f','').replace(u'\u3983','').replace(u'\u89c1','').replace(u'\u993b','').replace(u'\u5a7f','').replace(u'\u8d58','').replace(u'\u34e6','').replace(u'\u760b','').replace(u'\u7ecf','').replace(u'\u24b8','').replace(u'\xa6','').replace(u'\u3007','').replace(u'\u72ec','').replace(u'\u30fc','').replace(u'\xe9','').replace(u'\u30fb','').replace(u'\u9b47','').replace(u'\u68a6','').replace(u'\u8273','').replace(u'\u756b','').replace(u'\u65f6','').replace(u'\u2014','').replace(u'\u72bf','').replace(u'\u533b','').replace(u'\u25ac','').replace(u'\u5767','').replace(u'\u2219','').replace(u'\u7955','').replace(u'\u3f01','').replace(u'\xa9','').replace(u'\u5f72','').replace(u'\u95b6','').replace(u'\u25ab','').replace(u'\u2013','').replace(u'\u2609','').replace(u'\u880d','').replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6','')
                ]    
    
    
    return info_list
#.replace(u'\xf3','')


driver = webdriver.Chrome("C:/chromedriver.exe")
url="http://page.kakao.com"
driver.get(url)

button=driver.find_element_by_xpath('//*[@id="kpw-header"]/div/span/button')
button.click()
time.sleep(1)
driver.switch_to.window(driver.window_handles[-1])

idr=driver.find_element_by_xpath('//*[@id="loginEmailField"]/div/label')
idr.click()
idr=driver.find_element_by_xpath('//*[@id="id_email_2"]')
time.sleep(0.5)
idr.send_keys("narukane@naver.com")

pw=driver.find_element_by_xpath('//*[@id="login-form"]/fieldset/div[3]/label')
pw.click()
pw=driver.find_element_by_xpath('//*[@id="id_password_3"]')
time.sleep(0.5)
pw.send_keys("ksj3721*")

enter=driver.find_element_by_xpath('//*[@id="login-form"]/fieldset/div[8]/button[1]')
enter.click()
driver.switch_to.window(driver.window_handles[0])

time.sleep(2)

f = open('kakaopage_title_list6.csv','r', encoding='cp949')
f_w = open('kakaopage_result.csv', 'a', encoding='cp949', newline='')
wr = csv.writer(f_w)
csvReader = csv.reader(f)


# 첫 헤더 제외
next(csvReader)
count=0
for row in csvReader:
#    try:
    wr.writerow(kakaopage_crawling(row[0]))
#    except:
#        wr.writerow(kakaopage_crawling2(row[0]))
f_w.close()
f.close()

인생은 멜로
작품이름 : 인생은 멜로, 사랑은 에로
장르 : 로맨스
작가 : 이경하
완결여부 : 완결
이미지 : //dn-img-page.kakao.com/download/resource?kid=bDhaWz/hyikOfty9W/GI7xkqfvLuOvoag1vqXD00&filename=th1
15세 이용가
연재 수 : 40
최초 업데이트일 : 2018.01.04
출판사 : 정보 없음
작품소개
----------------------------------------
*이 작품은 15세 이용가로 재편집된 콘텐츠입니다.

평범한 여직원의 머릿속이 문란하다!
연이은 실연에 힘입어 ‘문란한 여자가 되리라’ 마음먹은 나이수.
방년 스물일곱의 나이에 타깃을 점찍었으니
오래전부터 다이어리의 절반을 차지하던 사내 프린스, 한기주다!
그래, 이참에 무미건조한 통나무에서 탈출하리라.
불감증? 이제는 아디오스(adiós). 
그런데 백마 탄 왕자님으로만 여겼던 그 남자가 이상하다.
다정한 미소 너머 사악한 꿍꿍이가 있는 듯한데…….

삶은 언제나 우연의 연속.
사람은 언제나 각기 다른 우주.
두 개의 우주가 각기 다른 사연과 생각으로 만나 인연을 엮어 나간다.
하지만 한 가지는 명심할 것!
어떤 고난과 역경이 닥치더라도 인생은 멜로처럼, 사랑은 에로처럼.
----------------------------------------
송은교
작품이름 : 송은교, 육체를 바꾸다
장르 : 로맨스
작가 : 김원경
완결여부 : 완결
이미지 : //dn-img-page.kakao.com/download/resource?kid=ckwU1k/hyikFJGUKu/cT8ZUZfktFGkj6pDcq9Y5K&filename=th1
15세 이용가
연재 수 : 23
최초 업데이트일 : 2017.12.29
출판사 : 정보 없음
작품소개
----------------------------------------
“제, 제가 누구라고요?”
“송은교라며.”
우현은 일부러 그녀의 반응을 살피기 위해 그녀를 그렇게 부른 것이

그 지독함
작품이름 : 사랑, 그 지독함
장르 : 로맨스
작가 : 유카
완결여부 : 완결
이미지 : //dn-img-page.kakao.com/download/resource?kid=nvzEm/hygsVHyPBD/2v1ax3F9MDoB4spK7UcnBK&filename=th1
15세 이용가
연재 수 : 71
최초 업데이트일 : 2017.10.23
출판사 : 정보 없음
작품소개
----------------------------------------
막연하게 끝이라고 생각했던 곳에서,
문득 그가 그리웠다.

인생이란 원래 계획대로 진행되는 법이 없다.
그를 손에서 놓아야 할 때가 왔다면, 바로 지금이라는 거다.
하루가 모자를 정도로 그 순간을 상상해왔다.

그가 어떤 사람인지가 중요한게 아니었다.
잘 알지도 못하는 그를 내가 얼마만큼이나 좋아하고 있는지가 중요했다.
----------------------------------------
그를 깨우다
작품이름 : 본능, 그를 깨우다
장르 : 로맨스
작가 : Benecia
완결여부 : 완결
이미지 : //dn-img-page.kakao.com/download/resource?kid=cbkvcs/hygsWUe2yE/G7gshJkcsAqKkTwrojXllK&filename=th1
15세 이용가
연재 수 : 37
최초 업데이트일 : 2017.10.20
출판사 : 정보 없음
작품소개
----------------------------------------
- 금. 동. 하 

미우그룹의 차남, 미우백화점의 사장 
사랑과 섹스는 게임이라 여기며, 자유로운 생활을 만끽하던 그에게 어느날 성큼 다가온 여자. 
그는 그녀에게 곁을 내주지 못했다. 불안해하던 그녀가 떠났다. 
그제야 동하는 그녀가 소중하다는 것을 알았다. 


- 문. 예. 진

설화그룹 현 회장의 동생인, 문 성주 사장의 딸 
대한민국 최고의 여배우로 주가를 날리던 시절, 돌연 금동하와 결혼하여 연예계를 은퇴했다. 
사랑하는 그가 나를 사랑하지 않는다는 것을 절

슬픔아
작품이름 : 슬픔아, 제발!
장르 : 로맨스
작가 : 명우
완결여부 : 완결
이미지 : //dn-img-page.kakao.com/download/resource?kid=McuAe/hyfOzeI8lM/aqo6LiZMBJeNkWgUmi5d1k&filename=th1
15세 이용가
연재 수 : 45
최초 업데이트일 : 2017.09.04
출판사 : 정보 없음
작품소개
----------------------------------------
복수를 위해 여자이길 포기한 그녀의 마지막 눈물

15년 전 가족이 살해당하던 날, 공포에 질린 채 혼자 도망쳐야 했던 수인. 그 후 매일같이 악몽에 시달리며 복수의 칼날을 벼려 왔던 그녀는 마침내 부모를 죽인 원수의 회사에 비서로 입사하게 된다. 하지만 입사한 회사의 주도권을 쥔 사람은 바로 원수의 아들 한선후. 면접 때부터 그에게 접근하고자 남장까지 했었던 그녀는 결국 입사한 지 5년 만에 그의 수행비서 자리를 차지하고, 그렇게 조금씩, 서서히 회사를 무너뜨릴 기회를 엿보며 긴장된 나날을 보낸다. 그러나, 뛰어난 외모에 오싹한 냉기. 마치 명계의 신 하데스를 연상케 하는 그는 그녀가 감히 쉽게 넘볼 수 있는 상대가 아니었는데….
----------------------------------------
밤의 안부
작품이름 : 밤의 안부 & 그대, 들리나요?(외전증보판)
장르 : 로맨스
작가 : 이사야
완결여부 : 완결
이미지 : //dn-img-page.kakao.com/download/resource?kid=eoLW6C/hygsW7tgco/nUFADHpxHkNln9VG1aPKsk&filename=th1
15세 이용가
연재 수 : 76
최초 업데이트일 : 2017.08.30
출판사 : 정보 없음
작품소개
----------------------------------------
[밤의 안부]

“좋아한다고 말해 주면 안 돼, 이예안?”
“좋아해….”
“사랑한다고 말해 줘.”
“…….”
그의 눈빛이 간절했다. 
“사랑… 인

In [14]:
import bs4
import requests
from urllib import parse
import selenium
import csv
import re
from selenium import webdriver
import time

def kakaopage_crawling(url):
    href=url
    driver.get(url)
    bs_obj = bs4.BeautifulSoup(driver.page_source, "html.parser")
    time.sleep(1)
    #pub_date=driver.find_element_by_xpath('//*[@id="root"]/div[3]/div/div/div[3]/ul/li[1]/div[2]/dl/dd').text
    #print("출간일 : " + pub_date) 
    #time.sleep(0.2)

    #if complete!="완결":
    #    update_days = driver.find_element_by_xpath('//*[@id="root"]/div[3]/div/div/div[1]/div[2]/div[2]/div[1]/p[1]')
    #    print("연재 요일 : " + update_days.text.replace(" 연재",""))

    #else:
    #    print("연재 요일 : 연재 종료")
    #time.sleep(0.2)
    title=driver.find_element_by_xpath('//*[@id="root"]/div[3]/div/div/div[1]/div[2]/div[1]/h2').text
    author=driver.find_element_by_xpath('//*[@id="root"]/div[3]/div/div/div[1]/div[2]/div[3]/div[1]/p[2]').text
    complete=driver.find_element_by_xpath('//*[@id="root"]/div[3]/div/div/div[1]/div[2]/div[3]/div[1]/p[1]/span').text
    img_src=bs_obj.find('div',{'class','jsx-3979090067 thumbnailBox'}).find('img').attrs['src']
    if(complete!='완결'):
        complete='미완결'
    
    total=driver.find_element_by_xpath('//*[@id="root"]/div[3]/div/div/div[3]/div[1]/span[1]')
    total=total.text.replace('전체(',"").replace(')',"")
    print("연재 수 : "+ total)
    time.sleep(0.2)

    pub_date=driver.find_element_by_xpath('//*[@id="root"]/div[3]/div/div/div[3]/ul/li[1]/div[2]/dl/dd').text
    print("최초 업데이트일 : " + pub_date)
    time.sleep(0.2)

    time.sleep(2)

    driver.find_element_by_xpath('//*[@id="root"]/div[3]/div/div/div[1]/div[2]/div[3]/div[2]/button[1]').click()
    #driver.set_window_size(1400,1000)
    publisher=None
    try:
        publisher=driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div/div/table/tbody/tr[1]/td[2]/div[3]/div[2]').text
    except:
        pass

    if publisher==None:
        publisher="정보 없음"
    print("출판사 : " + publisher)
    time.sleep(0.2)
    genre="로맨스"
    age=driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div/div/table/tbody/tr[1]/td[2]/div[4]/div[2]').text

    intro=driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div/div/table/tbody/tr[1]/td[2]/div[1]').text
    print("작품소개\n" + "----------------------------------------\n" + intro + "\n----------------------------------------")
    time.sleep(0.2)

    #driver2.quit()

    info_list = [title.replace(u'\u7448','').replace(u'\u2981','').replace(u'\u3e6e','').replace(u'\u586d','').replace(u'\u614f','').replace(u'\uff63','').replace(u'\uff62','').replace(u'\u2022','').replace(u'\u231f','').replace(u'\u231c','').replace(u'\xed','').replace(u'\u0165','').replace(u'\u0161','').replace(u'\xe1','').replace(u'\u2027','').replace(u'\u82e4','').replace(u'\u51e4','').replace(u'\u989c','').replace(u'\u971b','').replace(u'\u7978','').replace(u'\u6ce0','').replace(u'\u503e','').replace(u'\u98cb','').replace(u'\u22ef','').replace(u'\u554a','').replace(u'\u6720','').replace(u'\u71cf','').replace(u'\ufeff','').replace(u'\u5f37','').replace(u'\u630c','').replace(u'\u7681','').replace(u'\u512b','').replace(u'\u70e5','').replace(u'\u525f','').replace(u'\u3983','').replace(u'\u89c1','').replace(u'\u993b','').replace(u'\u5a7f','').replace(u'\u8d58','').replace(u'\u34e6','').replace(u'\u760b','').replace(u'\u7ecf','').replace(u'\u24b8','').replace(u'\xa6','').replace(u'\u3007','').replace(u'\u72ec','').replace(u'\u30fc','').replace(u'\xe9','').replace(u'\u30fb','').replace(u'\u9b47','').replace(u'\u68a6','').replace(u'\u8273','').replace(u'\u756b','').replace(u'\u65f6','').replace(u'\u2014','').replace(u'\u72bf','').replace(u'\u533b','').replace(u'\u25ac','').replace(u'\u5767','').replace(u'\u2219','').replace(u'\u7955','').replace(u'\u3f01','').replace(u'\xa9','').replace(u'\u5f72','').replace(u'\u95b6','').replace(u'\u25ab','').replace(u'\u2013','').replace(u'\u2609','').replace(u'\u880d','').replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 genre.replace(u'\u7448','').replace(u'\u2981','').replace(u'\u3e6e','').replace(u'\u586d','').replace(u'\u614f','').replace(u'\uff63','').replace(u'\uff62','').replace(u'\u2022','').replace(u'\u231f','').replace(u'\u231c','').replace(u'\xed','').replace(u'\u0165','').replace(u'\u0161','').replace(u'\xe1','').replace(u'\u2027','').replace(u'\u82e4','').replace(u'\u51e4','').replace(u'\u989c','').replace(u'\u971b','').replace(u'\u7978','').replace(u'\u6ce0','').replace(u'\u503e','').replace(u'\u98cb','').replace(u'\u22ef','').replace(u'\u554a','').replace(u'\u6720','').replace(u'\u71cf','').replace(u'\ufeff','').replace(u'\u5f37','').replace(u'\u630c','').replace(u'\u7681','').replace(u'\u512b','').replace(u'\u70e5','').replace(u'\u525f','').replace(u'\u3983','').replace(u'\u89c1','').replace(u'\u993b','').replace(u'\u5a7f','').replace(u'\u8d58','').replace(u'\u34e6','').replace(u'\u760b','').replace(u'\u7ecf','').replace(u'\u24b8','').replace(u'\xa6','').replace(u'\u3007','').replace(u'\u72ec','').replace(u'\u30fc','').replace(u'\xe9','').replace(u'\u30fb','').replace(u'\u9b47','').replace(u'\u68a6','').replace(u'\u8273','').replace(u'\u756b','').replace(u'\u65f6','').replace(u'\u2014','').replace(u'\u72bf','').replace(u'\u533b','').replace(u'\u25ac','').replace(u'\u5767','').replace(u'\u2219','').replace(u'\u7955','').replace(u'\u3f01','').replace(u'\xa9','').replace(u'\u5f72','').replace(u'\u95b6','').replace(u'\u25ab','').replace(u'\u2013','').replace(u'\u2609','').replace(u'\u880d','').replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 author.replace(u'\u7448','').replace(u'\u2981','').replace(u'\u3e6e','').replace(u'\u586d','').replace(u'\u614f','').replace(u'\uff63','').replace(u'\uff62','').replace(u'\u2022','').replace(u'\u231f','').replace(u'\u231c','').replace(u'\xed','').replace(u'\u0165','').replace(u'\u0161','').replace(u'\xe1','').replace(u'\u2027','').replace(u'\u82e4','').replace(u'\u51e4','').replace(u'\u989c','').replace(u'\u971b','').replace(u'\u7978','').replace(u'\u6ce0','').replace(u'\u503e','').replace(u'\u98cb','').replace(u'\u22ef','').replace(u'\u554a','').replace(u'\u6720','').replace(u'\u71cf','').replace(u'\ufeff','').replace(u'\u5f37','').replace(u'\u630c','').replace(u'\u7681','').replace(u'\u512b','').replace(u'\u70e5','').replace(u'\u525f','').replace(u'\u3983','').replace(u'\u89c1','').replace(u'\u993b','').replace(u'\u5a7f','').replace(u'\u8d58','').replace(u'\u34e6','').replace(u'\u760b','').replace(u'\u7ecf','').replace(u'\u24b8','').replace(u'\xa6','').replace(u'\u3007','').replace(u'\u72ec','').replace(u'\u30fc','').replace(u'\xe9','').replace(u'\u30fb','').replace(u'\u9b47','').replace(u'\u68a6','').replace(u'\u8273','').replace(u'\u756b','').replace(u'\u65f6','').replace(u'\u2014','').replace(u'\u72bf','').replace(u'\u533b','').replace(u'\u25ac','').replace(u'\u5767','').replace(u'\u2219','').replace(u'\u7955','').replace(u'\u3f01','').replace(u'\xa9','').replace(u'\u5f72','').replace(u'\u95b6','').replace(u'\u25ab','').replace(u'\u2013','').replace(u'\u2609','').replace(u'\u880d','').replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 complete.replace(u'\u7448','').replace(u'\u2981','').replace(u'\u3e6e','').replace(u'\u586d','').replace(u'\u614f','').replace(u'\uff63','').replace(u'\uff62','').replace(u'\u2022','').replace(u'\u231f','').replace(u'\u231c','').replace(u'\xed','').replace(u'\u0165','').replace(u'\u0161','').replace(u'\xe1','').replace(u'\u2027','').replace(u'\u82e4','').replace(u'\u51e4','').replace(u'\u989c','').replace(u'\u971b','').replace(u'\u7978','').replace(u'\u6ce0','').replace(u'\u503e','').replace(u'\u98cb','').replace(u'\u22ef','').replace(u'\u554a','').replace(u'\u6720','').replace(u'\u71cf','').replace(u'\ufeff','').replace(u'\u5f37','').replace(u'\u630c','').replace(u'\u7681','').replace(u'\u512b','').replace(u'\u70e5','').replace(u'\u525f','').replace(u'\u3983','').replace(u'\u89c1','').replace(u'\u993b','').replace(u'\u5a7f','').replace(u'\u8d58','').replace(u'\u34e6','').replace(u'\u760b','').replace(u'\u7ecf','').replace(u'\u24b8','').replace(u'\xa6','').replace(u'\u3007','').replace(u'\u72ec','').replace(u'\u30fc','').replace(u'\xe9','').replace(u'\u30fb','').replace(u'\u9b47','').replace(u'\u68a6','').replace(u'\u8273','').replace(u'\u756b','').replace(u'\u65f6','').replace(u'\u2014','').replace(u'\u72bf','').replace(u'\u533b','').replace(u'\u25ac','').replace(u'\u5767','').replace(u'\u2219','').replace(u'\u7955','').replace(u'\u3f01','').replace(u'\xa9','').replace(u'\u5f72','').replace(u'\u95b6','').replace(u'\u25ab','').replace(u'\u2013','').replace(u'\u2609','').replace(u'\u880d','').replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 img_src.replace(u'\u7448','').replace(u'\u2981','').replace(u'\u3e6e','').replace(u'\u586d','').replace(u'\u614f','').replace(u'\uff63','').replace(u'\uff62','').replace(u'\u2022','').replace(u'\u231f','').replace(u'\u231c','').replace(u'\xed','').replace(u'\u0165','').replace(u'\u0161','').replace(u'\xe1','').replace(u'\u2027','').replace(u'\u82e4','').replace(u'\u51e4','').replace(u'\u989c','').replace(u'\u971b','').replace(u'\u7978','').replace(u'\u6ce0','').replace(u'\u503e','').replace(u'\u98cb','').replace(u'\u22ef','').replace(u'\u554a','').replace(u'\u6720','').replace(u'\u71cf','').replace(u'\ufeff','').replace(u'\u5f37','').replace(u'\u630c','').replace(u'\u7681','').replace(u'\u512b','').replace(u'\u70e5','').replace(u'\u525f','').replace(u'\u3983','').replace(u'\u89c1','').replace(u'\u993b','').replace(u'\u5a7f','').replace(u'\u8d58','').replace(u'\u34e6','').replace(u'\u760b','').replace(u'\u7ecf','').replace(u'\u24b8','').replace(u'\xa6','').replace(u'\u3007','').replace(u'\u72ec','').replace(u'\u30fc','').replace(u'\xe9','').replace(u'\u30fb','').replace(u'\u9b47','').replace(u'\u68a6','').replace(u'\u8273','').replace(u'\u756b','').replace(u'\u65f6','').replace(u'\u2014','').replace(u'\u72bf','').replace(u'\u533b','').replace(u'\u25ac','').replace(u'\u5767','').replace(u'\u2219','').replace(u'\u7955','').replace(u'\u3f01','').replace(u'\xa9','').replace(u'\u5f72','').replace(u'\u95b6','').replace(u'\u25ab','').replace(u'\u2013','').replace(u'\u2609','').replace(u'\u880d','').replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 age.replace(u'\u7448','').replace(u'\u2981','').replace(u'\u3e6e','').replace(u'\u586d','').replace(u'\u614f','').replace(u'\uff63','').replace(u'\uff62','').replace(u'\u2022','').replace(u'\u231f','').replace(u'\u231c','').replace(u'\xed','').replace(u'\u0165','').replace(u'\u0161','').replace(u'\xe1','').replace(u'\u2027','').replace(u'\u82e4','').replace(u'\u51e4','').replace(u'\u989c','').replace(u'\u971b','').replace(u'\u7978','').replace(u'\u6ce0','').replace(u'\u503e','').replace(u'\u98cb','').replace(u'\u22ef','').replace(u'\u554a','').replace(u'\u6720','').replace(u'\u71cf','').replace(u'\ufeff','').replace(u'\u5f37','').replace(u'\u630c','').replace(u'\u7681','').replace(u'\u512b','').replace(u'\u70e5','').replace(u'\u525f','').replace(u'\u3983','').replace(u'\u89c1','').replace(u'\u993b','').replace(u'\u5a7f','').replace(u'\u8d58','').replace(u'\u34e6','').replace(u'\u760b','').replace(u'\u7ecf','').replace(u'\u24b8','').replace(u'\xa6','').replace(u'\u3007','').replace(u'\u72ec','').replace(u'\u30fc','').replace(u'\xe9','').replace(u'\u30fb','').replace(u'\u9b47','').replace(u'\u68a6','').replace(u'\u8273','').replace(u'\u756b','').replace(u'\u65f6','').replace(u'\u2014','').replace(u'\u72bf','').replace(u'\u533b','').replace(u'\u25ac','').replace(u'\u5767','').replace(u'\u2219','').replace(u'\u7955','').replace(u'\u3f01','').replace(u'\xa9','').replace(u'\u5f72','').replace(u'\u95b6','').replace(u'\u25ab','').replace(u'\u2013','').replace(u'\u2609','').replace(u'\u880d','').replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 total.replace(u'\u7448','').replace(u'\u2981','').replace(u'\u3e6e','').replace(u'\u586d','').replace(u'\u614f','').replace(u'\uff63','').replace(u'\uff62','').replace(u'\u2022','').replace(u'\u231f','').replace(u'\u231c','').replace(u'\xed','').replace(u'\u0165','').replace(u'\u0161','').replace(u'\xe1','').replace(u'\u2027','').replace(u'\u82e4','').replace(u'\u51e4','').replace(u'\u989c','').replace(u'\u971b','').replace(u'\u7978','').replace(u'\u6ce0','').replace(u'\u503e','').replace(u'\u98cb','').replace(u'\u22ef','').replace(u'\u554a','').replace(u'\u6720','').replace(u'\u71cf','').replace(u'\ufeff','').replace(u'\u5f37','').replace(u'\u630c','').replace(u'\u7681','').replace(u'\u512b','').replace(u'\u70e5','').replace(u'\u525f','').replace(u'\u3983','').replace(u'\u89c1','').replace(u'\u993b','').replace(u'\u5a7f','').replace(u'\u8d58','').replace(u'\u34e6','').replace(u'\u760b','').replace(u'\u7ecf','').replace(u'\u24b8','').replace(u'\xa6','').replace(u'\u3007','').replace(u'\u72ec','').replace(u'\u30fc','').replace(u'\xe9','').replace(u'\u30fb','').replace(u'\u9b47','').replace(u'\u68a6','').replace(u'\u8273','').replace(u'\u756b','').replace(u'\u65f6','').replace(u'\u2014','').replace(u'\u72bf','').replace(u'\u533b','').replace(u'\u25ac','').replace(u'\u5767','').replace(u'\u2219','').replace(u'\u7955','').replace(u'\u3f01','').replace(u'\xa9','').replace(u'\u5f72','').replace(u'\u95b6','').replace(u'\u25ab','').replace(u'\u2013','').replace(u'\u2609','').replace(u'\u880d','').replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 pub_date.replace(u'\u7448','').replace(u'\u2981','').replace(u'\u3e6e','').replace(u'\u586d','').replace(u'\u614f','').replace(u'\uff63','').replace(u'\uff62','').replace(u'\u2022','').replace(u'\u231f','').replace(u'\u231c','').replace(u'\xed','').replace(u'\u0165','').replace(u'\u0161','').replace(u'\xe1','').replace(u'\u2027','').replace(u'\u82e4','').replace(u'\u51e4','').replace(u'\u989c','').replace(u'\u971b','').replace(u'\u7978','').replace(u'\u6ce0','').replace(u'\u503e','').replace(u'\u98cb','').replace(u'\u22ef','').replace(u'\u554a','').replace(u'\u6720','').replace(u'\u71cf','').replace(u'\ufeff','').replace(u'\u5f37','').replace(u'\u630c','').replace(u'\u7681','').replace(u'\u512b','').replace(u'\u70e5','').replace(u'\u525f','').replace(u'\u3983','').replace(u'\u89c1','').replace(u'\u993b','').replace(u'\u5a7f','').replace(u'\u8d58','').replace(u'\u34e6','').replace(u'\u760b','').replace(u'\u7ecf','').replace(u'\u24b8','').replace(u'\xa6','').replace(u'\u3007','').replace(u'\u72ec','').replace(u'\u30fc','').replace(u'\xe9','').replace(u'\u30fb','').replace(u'\u9b47','').replace(u'\u68a6','').replace(u'\u8273','').replace(u'\u756b','').replace(u'\u65f6','').replace(u'\u2014','').replace(u'\u72bf','').replace(u'\u533b','').replace(u'\u25ac','').replace(u'\u5767','').replace(u'\u2219','').replace(u'\u7955','').replace(u'\u3f01','').replace(u'\xa9','').replace(u'\u5f72','').replace(u'\u95b6','').replace(u'\u25ab','').replace(u'\u2013','').replace(u'\u2609','').replace(u'\u880d','').replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''),
                 publisher.replace(u'\u7448','').replace(u'\u2981','').replace(u'\u3e6e','').replace(u'\u586d','').replace(u'\u614f','').replace(u'\uff63','').replace(u'\uff62','').replace(u'\u2022','').replace(u'\u231f','').replace(u'\u231c','').replace(u'\xed','').replace(u'\u0165','').replace(u'\u0161','').replace(u'\xe1','').replace(u'\u2027','').replace(u'\u82e4','').replace(u'\u51e4','').replace(u'\u989c','').replace(u'\u971b','').replace(u'\u7978','').replace(u'\u6ce0','').replace(u'\u503e','').replace(u'\u98cb','').replace(u'\u22ef','').replace(u'\u554a','').replace(u'\u6720','').replace(u'\u71cf','').replace(u'\ufeff','').replace(u'\u5f37','').replace(u'\u630c','').replace(u'\u7681','').replace(u'\u512b','').replace(u'\u70e5','').replace(u'\u525f','').replace(u'\u3983','').replace(u'\u89c1','').replace(u'\u993b','').replace(u'\u5a7f','').replace(u'\u8d58','').replace(u'\u34e6','').replace(u'\u760b','').replace(u'\u7ecf','').replace(u'\u24b8','').replace(u'\xa6','').replace(u'\u3007','').replace(u'\u72ec','').replace(u'\u30fc','').replace(u'\xe9','').replace(u'\u30fb','').replace(u'\u9b47','').replace(u'\u68a6','').replace(u'\u8273','').replace(u'\u756b','').replace(u'\u65f6','').replace(u'\u2014','').replace(u'\u72bf','').replace(u'\u533b','').replace(u'\u25ac','').replace(u'\u5767','').replace(u'\u2219','').replace(u'\u7955','').replace(u'\u3f01','').replace(u'\xa9','').replace(u'\u5f72','').replace(u'\u95b6','').replace(u'\u25ab','').replace(u'\u2013','').replace(u'\u2609','').replace(u'\u880d','').replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 intro.replace(u'\u7448','').replace(u'\u2981','').replace(u'\u3e6e','').replace(u'\u586d','').replace(u'\u614f','').replace(u'\uff63','').replace(u'\uff62','').replace(u'\u2022','').replace(u'\u231f','').replace(u'\u231c','').replace(u'\xed','').replace(u'\u0165','').replace(u'\u0161','').replace(u'\xe1','').replace(u'\u2027','').replace(u'\u82e4','').replace(u'\u51e4','').replace(u'\u989c','').replace(u'\u971b','').replace(u'\u7978','').replace(u'\u6ce0','').replace(u'\u503e','').replace(u'\u98cb','').replace(u'\u22ef','').replace(u'\u554a','').replace(u'\u6720','').replace(u'\u71cf','').replace(u'\ufeff','').replace(u'\u5f37','').replace(u'\u630c','').replace(u'\u7681','').replace(u'\u512b','').replace(u'\u70e5','').replace(u'\u525f','').replace(u'\u3983','').replace(u'\u89c1','').replace(u'\u993b','').replace(u'\u5a7f','').replace(u'\u8d58','').replace(u'\u34e6','').replace(u'\u760b','').replace(u'\u7ecf','').replace(u'\u24b8','').replace(u'\xa6','').replace(u'\u3007','').replace(u'\u72ec','').replace(u'\u30fc','').replace(u'\xe9','').replace(u'\u30fb','').replace(u'\u9b47','').replace(u'\u68a6','').replace(u'\u8273','').replace(u'\u756b','').replace(u'\u65f6','').replace(u'\u2014','').replace(u'\u72bf','').replace(u'\u533b','').replace(u'\u25ac','').replace(u'\u5767','').replace(u'\u2219','').replace(u'\u7955','').replace(u'\u3f01','').replace(u'\xa9','').replace(u'\u5f72','').replace(u'\u95b6','').replace(u'\u25ab','').replace(u'\u2013','').replace(u'\u2609','').replace(u'\u880d','').replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6',''), 
                 href.replace(u'\u7448','').replace(u'\u2981','').replace(u'\u3e6e','').replace(u'\u586d','').replace(u'\u614f','').replace(u'\uff63','').replace(u'\uff62','').replace(u'\u2022','').replace(u'\u231f','').replace(u'\u231c','').replace(u'\xed','').replace(u'\u0165','').replace(u'\u0161','').replace(u'\xe1','').replace(u'\u2027','').replace(u'\u82e4','').replace(u'\u51e4','').replace(u'\u989c','').replace(u'\u971b','').replace(u'\u7978','').replace(u'\u6ce0','').replace(u'\u503e','').replace(u'\u98cb','').replace(u'\u22ef','').replace(u'\u554a','').replace(u'\u6720','').replace(u'\u71cf','').replace(u'\ufeff','').replace(u'\u5f37','').replace(u'\u630c','').replace(u'\u7681','').replace(u'\u512b','').replace(u'\u70e5','').replace(u'\u525f','').replace(u'\u3983','').replace(u'\u89c1','').replace(u'\u993b','').replace(u'\u5a7f','').replace(u'\u8d58','').replace(u'\u34e6','').replace(u'\u760b','').replace(u'\u7ecf','').replace(u'\u24b8','').replace(u'\xa6','').replace(u'\u3007','').replace(u'\u72ec','').replace(u'\u30fc','').replace(u'\xe9','').replace(u'\u30fb','').replace(u'\u9b47','').replace(u'\u68a6','').replace(u'\u8273','').replace(u'\u756b','').replace(u'\u65f6','').replace(u'\u2014','').replace(u'\u72bf','').replace(u'\u533b','').replace(u'\u25ac','').replace(u'\u5767','').replace(u'\u2219','').replace(u'\u7955','').replace(u'\u3f01','').replace(u'\xa9','').replace(u'\u5f72','').replace(u'\u95b6','').replace(u'\u25ab','').replace(u'\u2013','').replace(u'\u2609','').replace(u'\u880d','').replace(u'\u7446','').replace(u'\u50fc','').replace(u'\u567a','').replace(u'\u6384','').replace(u'\u9556','').replace(u'\xa0','').replace(u'\u9de6','')
                ]   
    
    return info_list
#.replace(u'\uff63','')


driver = webdriver.Chrome("C:/chromedriver.exe")
url="http://page.kakao.com"
driver.get(url)

button=driver.find_element_by_xpath('//*[@id="kpw-header"]/div/span/button')
button.click()
time.sleep(1)
driver.switch_to.window(driver.window_handles[-1])

idr=driver.find_element_by_xpath('//*[@id="loginEmailField"]/div/label')
idr.click()
idr=driver.find_element_by_xpath('//*[@id="id_email_2"]')
time.sleep(0.5)
idr.send_keys("narukane@naver.com")

pw=driver.find_element_by_xpath('//*[@id="login-form"]/fieldset/div[3]/label')
pw.click()
pw=driver.find_element_by_xpath('//*[@id="id_password_3"]')
time.sleep(0.5)
pw.send_keys("ksj3721*")

enter=driver.find_element_by_xpath('//*[@id="login-form"]/fieldset/div[8]/button[1]')
enter.click()
driver.switch_to.window(driver.window_handles[0])

time.sleep(1.5)

f_w = open('kakaopage_result.csv', 'a', encoding='cp949', newline='')
wr = csv.writer(f_w)
#50501388,50450205
list=[50501388,50450205,50373607,50280432,50255388,50192711,50340267]
for i in range(len(list)):
    base_url='https://page.kakao.com/home?seriesId={}&orderby=asc'
    href=base_url.format(list[i])
    wr.writerow(kakaopage_crawling(href))

f_w.close()



연재 수 : 39
최초 업데이트일 : 2017.11.17
출판사 : 정보 없음
작품소개
----------------------------------------
사랑하는 사람을 잃고 더 이상 생화를
만질 수 없게 된 꽃집 주인, 하도연.
흐르는 시간 속에서 여전히 한겨울에 머물러 있는 그녀에게
“자, 오늘 치 적응입니다.”
생화, 메리골드를 선물하며 다가오는 남자, 구도경.

“메리골드. 이별의 슬픔, 가엾은 애정이랬던가요?”
처음에는 부담스러웠고,
“그것 말고 다른 꽃말도 있습니다.”
그다음엔 궁금했고,
“반드시 오고야 말 행복.”
이젠 그의 메리골드처럼 살고 싶어졌다.

“내가 정확하게 말했던가요?”
“뭐를요?”
“나는 아무래도 당신을 사랑하게 된 것 같습니다.”
갑작스러운 고백에 도연이 멍하게 그를 바라보았다.
도연의 가슴이 뛰었다.

그렇게 봄이 왔다.
----------------------------------------
연재 수 : 40
최초 업데이트일 : 2017.11.17
출판사 : 정보 없음
작품소개
----------------------------------------
“세상 다른 여자는 다 돼도 저는 절대 그럴 수 없어요.”

“아니 당신만큼 제격인 사람은 없어.”



자칭 장사꾼이라며 나타난 이강건설 CEO ‘이강재’ 

숨만 겨우 쉬며 죽은 듯 사는 지유에게 거절할 수 없는 제안을 하는 남자. 


“그 제안, 거절합니다.”

지유는 단호하게 일어섰다. 

“3년 전 그날…….” 　 
“그 이야기는……하고 싶지 않아요.”

그가 사악한 악마처럼 속삭였다. 

“한번이라도 생각해 봤나? 어째서 사고를 친 남자가 다른 누구도 아닌 약혼자의 친구일까? 그리고 왜 다량의 수면제를 복용했을까?”
“그건 술에 취해 일어난…….”
“정말 그렇게 생각해? 만에 하나 당신이 누명을 썼다면?” 

처음으로 그녀의 눈동자가 심하게 흔들리고 있었다.

“어때 이제는 구미가 당기나?”

[본 도서는 15세이용가에 맞게 수정&재편집된 도서입니다]
-----